In [1]:
#### Convert coffea output into root TH2's to feed into TUnfold
import coffea
import pickle
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import os
# import ROOT

In [2]:
from plugins import addFiles
files = ["coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2018.pkl", "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2017.pkl", "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2016.pkl"]
#files = ["coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2018.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2017.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2016.pkl"]
combinedFile = addFiles(files)
print(combinedFile)




coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjecALL.pkl


In [3]:
%matplotlib inline
fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjecALL.pkl"
# fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2018.pkl"
# fname = "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2018.pkl"
# fname ="coffeaOutput/trijetHistsTest_JetHT_pt200.0_eta2.4_bbloose.pkl"
eras = ["2016", "2017", "2018", "2016APV"]
dir = "rootFiles/"
for year in eras:
    print(year)
    if year in fname:
        print(year)
        year_str = year
        break
    else:
        year_str = "ALL"
with open(fname, "rb") as f:
    result = pickle.load( f )
axis_names = [ax.name for ax in result['jet_pt_mass_reco_u'].axes]
print(axis_names)
cats = [cat for cat in result['jet_pt_mass_reco_u'][{'ptreco':sum, 'dataset':sum, 'mreco':sum}].axes[0]]
print(cats)
if "JetHT" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsJetHT_jec_' + year_str + '.root'
    elif "trijet" in fname:
        rootfname = 'trijetHistsJetHT_jec_' + year_str + '.root'
elif "QCD" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsQCDsim_jec_' + year_str + '.root'
    # integrate and sum over axes
    elif "trijet" in fname:
        rootfname = 'trijetHistsQCDsim_jec' + year_str + '.root'
if os.path.exists(dir + rootfname):
    rootfile = uproot.recreate(dir+rootfname)
else:
    rootfile = uproot.create(dir+rootfname)
    #### only add hists containing gen if SIM files
response_entries = {}
if "QCD" in fname:
    for syst in cats:
        rootfile['ptgen_mgen_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['ptgen_mgen_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptgen","mgen")
        rootfile['ptreco_mreco_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco")
        rootfile['ptreco_mreco_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptreco","mreco")
        rootfile['misses_ptgen_mgen_'+syst] = result['misses'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['fakes_ptreco_mreco_'+syst] = result['fakes'][{'syst':syst}].project("ptreco","mreco")
        response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
        response_matrix_g_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges= result['response_matrix_g'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
        print(response_matrix_g_values.shape)
        ptreco_centers = (ptreco_edges[:-1]+ptreco_edges[1:])/2
        print(ptreco_centers[np.newaxis])
        mreco_centers = (mreco_edges[:-1]+mreco_edges[1:])/2
        ptgen_centers = (ptgen_edges[:-1]+ptgen_edges[1:])/2
        mgen_centers = (mgen_edges[:-1]+mgen_edges[1:])/2
        print(np.shape(response_matrix_g_values))
        print(np.shape(response_matrix_g_values[np.newaxis]))
        response_entries.update({'groomed_'+syst:response_matrix_g_values[np.newaxis],
                           'ungroomed_'+syst:response_matrix_u_values[np.newaxis]})
    rootfile['response']=response_entries
else:
    for syst in cats:
        rootfile['ptreco_mreco_u_'+syst] = result['jet_pt_mass_reco_u'][{'syst':syst}].project("ptreco", "mreco")
        rootfile['ptreco_mreco_g_'+syst] = result['jet_pt_mass_reco_g'][{'syst':syst}].project("ptreco","mreco")
rootfile.close()

2016
2017
2018
2016APV
['dataset', 'syst', 'ptreco', 'mreco']
['nominal', 'jesUp', 'jesDown', 'jerUp', 'jerDown']
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)


In [4]:
fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjecALL.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
print(result.keys())
response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
print(response_matrix_g_values.shape)
response_matrix_g_values= result['response_matrix_g'][{'syst':'nominal'}].project("ptreco", "mreco", "ptgen", "mgen").values()
nptreco,nmassreco,nptgen,nmassgen = response_matrix_g_values.shape
response_matrix_g_final = response_matrix_g_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) ) 
response_matrix_u_final = response_matrix_u_values.reshape( (nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2) ) 
print(response_matrix_g_final.shape)
response_matrix_g_final2 = result['response_matrix_g'][{'syst':"nominal"}].project("ptreco", "mreco", "ptgen", "mgen").values(flow=True).reshape( (nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2))
print(np.sum(response_matrix_g_final[26,:]))
print(np.sum(response_matrix_g_final2[26,:]))
print(np.sum(response_matrix_u_final[26,:]))

dict_keys(['jet_mass', 'jet_pt', 'jet_eta', 'btag', 'cutflow', 'njet_gen', 'dphimin_gen', 'asymm_gen', 'njet_reco', 'dphimin_reco', 'asymm_reco', 'jet_dr_reco_gen', 'jet_mass_reco_over_gen', 'jet_pt_reco', 'jet_pt_gen', 'jet_pt_reco_over_gen', 'jet_eta_reco', 'jet_eta_gen', 'jet_rap_reco', 'jet_rap_gen', 'jet_dphi_reco', 'jet_ptasymm_reco', 'jet_dr_gen_subjet', 'jet_dr_reco_to_gen_subjet', 'jet_sd_mass_reco', 'jet_sd_mass_gen', 'misses_g', 'fakes_g', 'misses', 'fakes', 'jet_pt_mass_reco_u', 'jet_pt_mass_reco_g', 'jet_pt_mass_gen_u', 'jet_pt_mass_gen_g', 'jet_m_pt_u_reco_over_gen', 'jet_m_pt_g_reco_over_gen', 'response_matrix_u', 'response_matrix_g', 'weights', 'systematics'])
(12, 26, 12, 14)
(240, 120)
1750.0
763.0
5.596834033787367


In [5]:
# fname = uproot.open('root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/42B4DEA0-C0E7-A944-80DC-1D107A0F63FB.root')

In [6]:
# print(fname["Events"].keys("*HTXS*"))

In [7]:
hists = uproot.open('rootFiles/trijetHistsQCDsim_jecALL.root')
print(hists.keys())
hist_vals_g, hist_edges, hist_widths = hists['ptreco_mreco_g_nominal'].to_numpy(flow=True)
hist_vals_u, hist_edges, hist_widths = hists['ptreco_mreco_u_nominal'].to_numpy(flow=True)
print(hist_vals_g[1][0])
print(hist_vals_u[1][0])

['ptgen_mgen_u_nominal;1', 'ptgen_mgen_g_nominal;1', 'ptreco_mreco_u_nominal;1', 'ptreco_mreco_g_nominal;1', 'misses_ptgen_mgen_nominal;1', 'fakes_ptreco_mreco_nominal;1', 'ptgen_mgen_u_jesUp;1', 'ptgen_mgen_g_jesUp;1', 'ptreco_mreco_u_jesUp;1', 'ptreco_mreco_g_jesUp;1', 'misses_ptgen_mgen_jesUp;1', 'fakes_ptreco_mreco_jesUp;1', 'ptgen_mgen_u_jesDown;1', 'ptgen_mgen_g_jesDown;1', 'ptreco_mreco_u_jesDown;1', 'ptreco_mreco_g_jesDown;1', 'misses_ptgen_mgen_jesDown;1', 'fakes_ptreco_mreco_jesDown;1', 'ptgen_mgen_u_jerUp;1', 'ptgen_mgen_g_jerUp;1', 'ptreco_mreco_u_jerUp;1', 'ptreco_mreco_g_jerUp;1', 'misses_ptgen_mgen_jerUp;1', 'fakes_ptreco_mreco_jerUp;1', 'ptgen_mgen_u_jerDown;1', 'ptgen_mgen_g_jerDown;1', 'ptreco_mreco_u_jerDown;1', 'ptreco_mreco_g_jerDown;1', 'misses_ptgen_mgen_jerDown;1', 'fakes_ptreco_mreco_jerDown;1', 'response;1']
763.0
5.596834033787367


In [10]:
hists = uproot.open('rootFiles/trijetHistsQCDsim_jecALL.root')
print(hists['response'].keys())
resp_groomed = hists['response']['groomed_nominal'].array().to_numpy()[0].reshape((nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2))
print(np.sum(resp_groomed[26,:]))

['groomed_nominal', 'ungroomed_nominal', 'groomed_jesUp', 'ungroomed_jesUp', 'groomed_jesDown', 'ungroomed_jesDown', 'groomed_jerUp', 'ungroomed_jerUp', 'groomed_jerDown', 'ungroomed_jerDown']
763.0
